In [ ]:
import numpy as np

def get_theta_phase(LFP, spike_times, sampling_rate, peakFreq, filtHalfBandWidth=3, powerThresh=5):
    # Filter the LFP around the peakFreq
    nyq = 0.5 * sampling_rate
    low = (peakFreq - filtHalfBandWidth) / nyq
    high = (peakFreq + filtHalfBandWidth) / nyq
    taps = firwin(round(sampling_rate)+1, [low, high], pass_zero=False, window='blackman', fs=sampling_rate)
    theta_LFP = filtfilt(taps, 1, LFP)

    # Get the instantaneous phase using Hilbert transform
    analytic_signal = hilbert(theta_LFP)
    eegPhase = np.angle(analytic_signal)
    eegPhase = np.mod(eegPhase, 2*np.pi)  # wrap into 0-2pi range

    # Find phase transitions (i.e., cycle boundaries)
    phaseTrans = np.diff(eegPhase) < -np.pi
    phaseTrans = np.hstack(([True], phaseTrans, [True]))

    # Remove phase slips
    phaseSlips = np.hstack(([False], np.diff(np.unwrap(eegPhase)) < 0, [False]))
    phaseTrans[phaseSlips] = False

    # Numeric cycle index
    cycleN = np.cumsum(phaseTrans[:-1])  # Exclude the last element to match the LFP's length

    # EEG power and power per cycle
    power = theta_LFP**2
    powerPerCycle = np.bincount(cycleN, power) / np.bincount(cycleN)
    
    # Remove bad data based on power and cycle length thresholds
    thresh = np.percentile(powerPerCycle, powerThresh)
    badPowerCycle = np.where(powerPerCycle < thresh)[0]
    badPowerInd = np.isin(cycleN, badPowerCycle)
    
    cycleLength = np.bincount(cycleN)
    passBand = [1/(peakFreq + filtHalfBandWidth), 1/(peakFreq - filtHalfBandWidth)]
    cycleLengthLim = np.ceil(1 / np.array(passBand) * sampling_rate)
    badLengthCycle = np.where((cycleLength < cycleLengthLim[1]) | (cycleLength > cycleLengthLim[0]))[0]
    badLengthInd = np.isin(cycleN, badLengthCycle)
    
    # Mask bad indices
    badIndices = badLengthInd | badPowerInd
    eegPhase[badIndices] = np.nan

    # Map spike times to phase
    spike_indices = (spike_times * sampling_rate).astype(int)
    spike_phases = eegPhase[spike_indices]

    return spike_phases




In [1]:
## Load spikes from t-maze trials
# Set system path to allow ephys class import
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from ephys import *
from ephys_utils import *

obj = ephys(recording_type = 'nexus', path = '/home/isabella/Documents/isabella/jake/recording_data/r1364/2023-06-14')

obj.load_spikes('good')

# Select only spikes from t-maze trials
t_maze_trials = [i for i, s in enumerate(obj.trial_list) if 't-maze' in s]
t_maze_spikes = select_spikes_by_trial(obj.spike_data, t_maze_trials)

print(f"{len(t_maze_spikes['spike_times'])} spikes loaded from {len(np.unique(t_maze_spikes['spike_clusters']))} clusters in {len(t_maze_trials)} t-maze trial(s)")

133373 spikes loaded from 20 clusters in 2 t-maze trial(s)


In [ ]:
## Load LFP for each channel with good units

# Get unique channels where good units are found
good_channels = list(sorted(set(obj.spike_data['cluster_channels'].values())))
good_channels = [str(i) for i in good_channels]

# Load LFP trace for each channel
for i in t_maze_trials:
    obj.load_lfp(i, 1250, 0, 600, good_channels)

'<generator object <genexpr> at 0x7efca2e43f50>'